# 인스타그램 로그인
<hr>
아래에 정의된 함수들을 한 번씩만 실시하고, 이 함수를 사용한 코드들을 모아둔 코드에서 크롤링을 실시하기
위의 코드들은 함수를 미리 정의하여, 크롤링 사용에 용이하도록 만든 코드들임

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 크롬 브라우저 열기
driver = webdriver.Chrome('C:/Temp/chromedriver')

driver.get("https://www.instagram.com/accounts/login/")
time.sleep(2)

email = '본인 ID'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '본인 PWD' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

In [6]:
# 인스타그램 해시태그 검색

def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
import time

word = "산수갑산"
url = insta_searching(word)
driver.get(url)

In [ ]:
# 총 게시물 개수

postnumber = driver.find_elements_by_css_selector("span > span.g47SY")[0].text.replace(',', '')
postnumber = int(postnumber)

In [ ]:
# 게시물 내용 크롤링

import re
from bs4 import BeautifulSoup

def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    try:
        tags = re.findall(r'#[^\s#,\\]+', content)  
    except:
        tags = ' '
    # 작성일자 
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    except:
        date = ''
    # 좋아요 
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # 위치
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
   
    data = [content, date, like, place, tags]
    time.sleep(1)
    return data
    

get_content(driver)

In [ ]:
# 다음 게시물로 이동

def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()

move_next(driver)

# 인스타그램 게시물 크롤링 하기    <img src='data/instagram.PNG' width=120 height=120 align="right">         
### : 본격적인 크롤링은 아래의 코드를 사용하기
<hr>
위에 코드들을 한 번 씩 실행 후, 아래의 코드에 word 내용만 변경하여 실행하면 반복 실행 

In [ ]:
import pandas as pd

df = pd.read_excel('output/음식점 모음.xlsx').fillna("missing")

for i in range(30) :
    for j in range(20) :
        if df.iloc[i, j] != "missing":
            search = df.iloc[i, j].replace(' ', '')

            url = insta_searching(search)
            driver.get(url)
            time.sleep(8)

            postnumber = driver.find_elements_by_css_selector("span > span.g47SY")[0].text.replace(',', '')
            postnumber = int(postnumber)

            select_first(driver)
            instagram = []
            data = []

            for k in range(200) :
                html = driver.page_source
                soup = BeautifulSoup(html, 'lxml')
                # 본문 내용 
                try:
                    content = soup.select('div.C4VMK > span')[0].text
                except:
                    content = ' '
                # 해시태그 
                try:
                    tags = re.findall(r'#[^\s#,\\]+', content)  
                except:
                    tags = ' '
                # 작성일자 
                try:
                    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
                except:
                    date = ''
                # 좋아요 
                try:
                    like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
                except:
                    like = 0
                # 위치
                try: 
                    place = soup.select('div.M30cS')[0].text
                except:
                    place = ''

                data = [content, date, like, place, tags]
                instagram = (instagram,data,'\n')
                time.sleep(0.5)
                move_next(driver)
                time.sleep(2)
           
            data_df = pd.DataFrame(instagram)
            data_df.columns = ['content','hashtag','date','like','place']
            data_df.to_excel('./data/instagram/'+ search +'.csv', encoding="utf-8")